In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.environ.get('LANGCHAIN_API_KEY')
os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY')

In [16]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///patients.db")
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT sql FROM sqlite_master WHERE tbl_name in ('Patient') AND type = 'table';"))
print(db.run("SELECT * from Patient limit 10;"))

sqlite
['BedUtilization', 'HealthCareProvider', 'HospitalDepartment', 'Patient', 'PatientAdmissionHistory', 'StaffUtilization']
[('CREATE TABLE Patient (EmergencyContactPhone TEXT, EmergencyContactName TEXT, ChronicConditions TEXT, KnownDrugAllergies TEXT, InsuranceProvider TEXT, PrimaryCarePhysician TEXT, NationalID TEXT, Address_AddressLine1 TEXT, Address_State TEXT, Address_Country TEXT, Address_ZipCode TEXT, Email TEXT,...',)]
[('8418601623', 'Sarah Williams', 'Hypertension', 'Sulfa Drugs', 'Harrington PLC', 'Michael Choi', '496-34-4700', '046 Carolyn Crest', 'NJ', 'USA', '14328', 'perryrachel@le.info', '5138062224', 'Male', '1960-02-08', 'King', 'Timothy', '51127'), ('2939404640', 'Jeffrey White', 'None', 'None', 'Flowers Ltd', 'Mary Quinn', '269-90-9934', '0732 Shawn Centers Apt. 075', 'NJ', 'USA', '14328', 'boyerjacqueline@chan.biz', '7770894315', 'Other', '1947-07-21', 'Smith', 'Michael', '84092'), ('7166129064', 'Marvin Santiago', 'Heart Disease', 'None', 'Mitchell-Lewis', 'Sa

In [4]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [6]:
# from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

# toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [7]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)

In [8]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")


table_names = "\n".join(db.get_usable_table_names())
system = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_names}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{input}"),
    ]
)

llm_with_tools = llm.bind_tools([Table])
output_parser = PydanticToolsParser(tools=[Table])

system = """You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
Return the names of any SQL tables that are relevant to the user question.
Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

The tables are:

Music
Business
Hospital
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{input}"),
    ]
)

category_chain = prompt | llm_with_tools | output_parser
#category_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

In [9]:
from typing import List


def get_tables(categories: List[Table]) -> List[str]:
    tables = []
    for category in categories:
        if category.name == "Music":
            tables.extend(
                [
                    "Album",
                    "Artist",
                    "Genre",
                    "MediaType",
                    "Playlist",
                    "PlaylistTrack",
                    "Track",
                ]
            )
        elif category.name == "Business":
            tables.extend(["Customer", "Employee", "Invoice", "InvoiceLine"])
        elif category.name == "Hospital":
            tables.extend(["Patient","patients"])    
    return tables


table_chain = category_chain | get_tables
#table_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

In [10]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
system = """You are a {dialect} expert. Given an input question, create a syntactically correct {dialect} query to run.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per {dialect}. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Only use the following tables:
{table_info}

Write an initial draft of the query. Then double check the {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query.Do not include characters ` in the beginning and end of the sql.
Remove any unwanted characters like back ticks :,;,` in the beginning and end.
If there are no mistakes, just reproduce the original query with no further commentary.

Use format:

First draft: <<FIRST_DRAFT_QUERY>>
Final answer: <<FINAL_ANSWER_QUERY>>
"""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}")]
).partial(dialect=db.dialect)


def parse_final_answer(output: str) -> str:
    return output.split("Final answer: ")[1]


full_chain = create_sql_query_chain(llm, db, prompt=prompt) | parse_final_answer

execute_query = QuerySQLDataBaseTool(db=db,use_query_checker=True,response_format='content')
write_query = full_chain
chain = write_query | execute_query

In [11]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

table_chain = {"input": itemgetter("question")} | table_chain

chain = (
    RunnablePassthrough.assign(query=write_query,table_names_to_use=table_chain).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)



In [13]:
response = chain.invoke(
                            {"question": "Address of Michael Choi"}
                        )
print(response)

The address of Michael Choi is 046 Carolyn Crest, NJ, USA, 14328.
